# EmploymentPy
---

## Python ETL / data engineering code for city-level-employment project (project #3)
### Pulls unemployment data via API from the Bureau of Labor Statistics (BLS), scrapes lat/long coordinates and population data from 

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import datetime as dt

from splinter import Browser
from bs4 import BeautifulSoup
from io import StringIO

import requests
import urllib.parse as up
import json
import csv

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

# Import API key for Bureau of Labor Statistics (BLS) API 2.0
from api_keys import bls_api_key

In [2]:
# read US Cities data file and load key columns into DF

city_list = []
lat_list = []
long_list = []
pop_list = []


with open('input_data/us_cities_loc_pop.csv') as city_data_file:
        
    city_data = csv.reader(city_data_file)


    for row in city_data:
        
        if row[0] != 'city':
            city_list.append(row[0])
        
        if row[6] != 'lat':
            lat_list.append(row[6])

        if row[7] != 'lng':
            long_list.append(row[7])
            
        if row[8] != 'population':
            pop_list.append(row[8])
            


city_dict = {'City':city_list, 'Latitude':lat_list, 'Longitude':long_list, 'Population':pop_list}


In [3]:

city_loc_pop_df = pd.DataFrame(city_dict)

city_loc_pop_df.head(100)


,City,Latitude,Longitude,Population
0,New York,40.6943,-73.9249,18972871
1,Los Angeles,34.1141,-118.4068,12121244
2,Chicago,41.8375,-87.6866,8595181
3,Miami,25.7840,-80.2101,5711945
4,Dallas,32.7935,-96.7667,5668165
...,...,...,...,...
95,Long Beach,33.7977,-118.1670,466565
96,Reno,39.5497,-119.8483,463328
97,Madison,43.0822,-89.3930,461778
98,Little Rock,34.7256,-92.3577,457379


In [4]:
# create DF of key metric codes+names

metric_dict = {'Metric Code':['03','04','05','06','07','08','09'], 'Metric Name':['Unemployment Rate','Unemployment','Employment','Labor Force','Employment-Population Ratio','Labor Force Participation Rate','Civilian Noninstitutional Population']}

metric_df = pd.DataFrame(metric_dict)

metric_df.head(10)


,Metric Code,Metric Name
0,03,Unemployment Rate
1,04,Unemployment
2,05,Employment
3,06,Labor Force
4,07,Employment-Population Ratio
5,08,Labor Force Participation Rate
6,09,Civilian Noninstitutional Population


In [5]:

# browser = Browser('chrome')

# area_codes_url = 'https://download.bls.gov/pub/time.series/la/la.area'
# browser.visit(area_codes_url)

# html = browser.html
# soup = BeautifulSoup(html, 'html.parser')

# print(html)

# site_text = soup.find('pre').text

# print(site_text)


In [6]:

browser = Browser('chrome')

area_codes_url = 'https://download.bls.gov/pub/time.series/la/la.area'
browser.visit(area_codes_url)

html_text = browser.html
soup = BeautifulSoup(html_text, 'html.parser')

site_text = soup.find('pre').text

print(site_text)


area_type_code	area_code	area_text	display_level	selectable	sort_sequence
A	ST0100000000000	Alabama	0	T	1
A	ST0200000000000	Alaska	0	T	149
A	ST0400000000000	Arizona	0	T	193
A	ST0500000000000	Arkansas	0	T	257
A	ST0600000000000	California	0	T	383
A	ST0800000000000	Colorado	0	T	772
A	ST0900000000000	Connecticut	0	T	908
A	ST1000000000000	Delaware	0	T	1100
A	ST1100000000000	District of Columbia	0	T	1112
A	ST1200000000000	Florida	0	T	1119
A	ST1300000000000	Georgia	0	T	1331
A	ST1500000000000	Hawaii	0	T	1585
A	ST1600000000000	Idaho	0	T	1595
A	ST1700000000000	Illinois	0	T	1676
A	ST1800000000000	Indiana	0	T	1961
A	ST1900000000000	Iowa	0	T	2142
A	ST2000000000000	Kansas	0	T	2304
A	ST2100000000000	Kentucky	0	T	2454
A	ST2200000000000	Louisiana	0	T	2626
A	ST2300000000000	Maine	0	T	2730
A	ST2400000000000	Maryland	0	T	3302
A	ST2500000000000	Massachusetts	0	T	3349
A	ST2600000000000	Michigan	0	T	3747
A	ST2700000000000	Minnesota	0	T	3972
A	ST2800000000000	Mississippi	0	T	4144
A	ST2900000000000	Missouri	0	

In [13]:

area_codes_df = pd.read_table(StringIO(site_text), delimiter = '\t')
area_codes_df.head()


,area_type_code,area_code,area_text,display_level,selectable,sort_sequence
0,A,ST0100000000000,Alabama,0,T,1
1,A,ST0200000000000,Alaska,0,T,149
2,A,ST0400000000000,Arizona,0,T,193
3,A,ST0500000000000,Arkansas,0,T,257
4,A,ST0600000000000,California,0,T,383


In [24]:
# Narrow down to only rows with Area Type Code = 'G' ("Cities and towns above 25,000 population"), per https://download.bls.gov/pub/time.series/la/la.area_type

city_area_codes_df = area_codes_df[area_codes_df['area_type_code']=='G'].reset_index()

city_area_codes_df.drop(columns=['index', 'display_level', 'selectable', 'sort_sequence', 'area_type_code'], inplace=True)
city_area_codes_df.rename(columns={'area_code': 'City Area Code', 'area_text': 'City'}, inplace=True)

print(f"{city_area_codes_df['City Area Code'].size} total records of Area Type 'G': 'Cities and towns above 25,000 population'")

city_area_codes_df.head()


1940 total records of Area Type 'G': 'Cities and towns above 25,000 population'


,City Area Code,City
0,CS0907310000000,"Branford town, CT"
1,CS0914160000000,"Cheshire town, CT"
2,CS0922630000000,"East Hartford town, CT"
3,CS0922910000000,"East Haven town, CT"
4,CS0925990000000,"Enfield town, CT"


In [25]:

city_name_word_count = []
split_city_name = []

for city in city_area_codes_df['City']:
    city_name_word_count.append(len(city.split()))
    split_city_name.append(city.split())

city_area_codes_df['city_name_word_count'] = city_name_word_count
city_area_codes_df['split_city_name'] = split_city_name

city_area_codes_df.head()


,City Area Code,City,city_name_word_count,split_city_name
0,CS0907310000000,"Branford town, CT",3,"[Branford, town,, CT]"
1,CS0914160000000,"Cheshire town, CT",3,"[Cheshire, town,, CT]"
2,CS0922630000000,"East Hartford town, CT",4,"[East, Hartford, town,, CT]"
3,CS0922910000000,"East Haven town, CT",4,"[East, Haven, town,, CT]"
4,CS0925990000000,"Enfield town, CT",3,"[Enfield, town,, CT]"


In [31]:

new_city_list = []
type_list = []
state_list = []


city_series = city_area_codes_df['City']
city_word_count_series = city_area_codes_df['city_name_word_count']
split_city_series = city_area_codes_df['split_city_name']
total_rows = city_series.size


for i in range(total_rows):
    
    city = city_series[i]
    word_count = city_word_count_series[i]
    split_city = split_city_series[i]
    
    new_city = ''
    
    for j in range(word_count-2):
        
        if j == 0:
            new_city += split_city[j]
        else:
            new_city += " " + split_city[j]
    
    new_city_list.append(new_city)
    type_list.append(split_city[word_count-2].replace(",",""))
    state_list.append(split_city[word_count-1])
        
city_area_codes_df['City'] = new_city_list
city_area_codes_df['Type'] = type_list
city_area_codes_df['State'] = state_list

city_area_codes_df.head()


,City Area Code,City,city_name_word_count,split_city_name,Type,State
0,CS0907310000000,Branford,3,"[Branford, town,, CT]",town,CT
1,CS0914160000000,Cheshire,3,"[Cheshire, town,, CT]",town,CT
2,CS0922630000000,East Hartford,4,"[East, Hartford, town,, CT]",town,CT
3,CS0922910000000,East Haven,4,"[East, Haven, town,, CT]",town,CT
4,CS0925990000000,Enfield,3,"[Enfield, town,, CT]",town,CT


In [34]:

city_codes_df = city_area_codes_df[city_area_codes_df['Type']=='city'].reset_index()

city_codes_df.drop(columns=['city_name_word_count', 'split_city_name', 'Type', 'index'], inplace=True)

print(f"{city_codes_df['City Area Code'].size} total records of Type 'city', subset of all records with Area Type 'G'")

city_codes_df.head()


1479 total records of Type 'city', subset of all records with Area Type 'G'


,City Area Code,City,State
0,CT0100820000000,Alabaster,AL
1,CT0101852000000,Anniston,AL
2,CT0102956000000,Athens,AL
3,CT0103076000000,Auburn,AL
4,CT0105980000000,Bessemer,AL


In [ ]:
>>> df["Suffix"] = df["Story"].str.split().str[-1]

len(str.split())

In [32]:

cities_split_df = city_area_codes_df['City'].str.split(' ', expand=True)

area_codes_df = city_area_codes_df.join(cities_split_df, how='inner', lsuffix='_df1', rsuffix='_df2')

area_codes_df.head()


,City Area Code,City,0,1,2,3,4,5,6,7
0,CS0907310000000,"Branford town, CT",Branford,"town,",CT,None,None,None,None,None
1,CS0914160000000,"Cheshire town, CT",Cheshire,"town,",CT,None,None,None,None,None
2,CS0922630000000,"East Hartford town, CT",East,Hartford,"town,",CT,None,None,None,None
3,CS0922910000000,"East Haven town, CT",East,Haven,"town,",CT,None,None,None,None
4,CS0925990000000,"Enfield town, CT",Enfield,"town,",CT,None,None,None,None,None


In [ ]:
area_type_code_list = []
area_code_list = []
area_text_list = []

for row in area_codes_df:

    new_row = []
    
    for item in row:
        
        new_row.append = item.split()
        newer_row_length = len(newer_row)
        
        print(f"newer_row = {newer_row}; newer_row_length = {newer_row_length}")
        
        area_type_code_list.append(newer_row[0])
        area_code_list.append(newer_row[1])
        
        area_text = ''
        
        for x in range(newer_row_length-7):
            
            next_area_text_bit = newer_row[x+2]
            
            if area_text == '':
                area_text = next_area_text_bit
            else:
                area_text += ' ' + next_area_text_bit
        
        
        area_text_list.append(area_text)


area_codes_df['Area Type Code'] = area_type_code_list
area_codes_df['City Area Code'] = area_code_list
area_codes_df['City'] = area_text_list


area_codes_df.head(50)


In [4]:
################################################################################

with open('input_data/area_codes.csv') as area_codes_file:
        
    area_codes_data = csv.reader(area_codes_file, delimiter = '\t')
    
    area_codes_df = pd.DataFrame(area_codes_data, columns=['Raw Data'])
    
area_codes_df.head(10)


,Raw Data
0,﻿area_type_code area_code area_text ...
1,A ST0100000000000 Alabama 0 T 1
2,A ST0200000000000 Alaska 0 T ...
3,A ST0400000000000 Arizona 0 T ...
4,A ST0500000000000 Arkansas 0 ...
5,A ST0600000000000 California 0 ...
6,A ST0800000000000 Colorado 0 ...
7,A ST0900000000000 Connecticut 0 ...
8,A ST1000000000000 Delaware 0 ...
9,A ST1100000000000 District of Columbia ...


In [7]:
################################################################################

area_type_code_list = []
area_code_list = []
area_text_list = []

for row in area_codes_df['Raw Data']:

    new_row = row.split('\t')

    
    for item in new_row:
        
        newer_row = item.split()
        newer_row_length = len(newer_row)
        
        print(f"newer_row = {newer_row}; newer_row_length = {newer_row_length}")
        
        area_type_code_list.append(newer_row[0])
        area_code_list.append(newer_row[1])
        
        area_text = ''
        
        for x in range(newer_row_length-7):
            
            next_area_text_bit = newer_row[x+2]
            
            if area_text == '':
                area_text = next_area_text_bit
            else:
                area_text += ' ' + next_area_text_bit
        
        
        area_text_list.append(area_text)


area_codes_df['Area Type Code'] = area_type_code_list
area_codes_df['City Area Code'] = area_code_list
area_codes_df['City'] = area_text_list


area_codes_df.head(50)


newer_row = ['\ufeffarea_type_code', 'area_code', 'area_text', 'display_level', 'selectable', 'sort_sequence']; newer_row_length = 6
newer_row = ['A', 'ST0100000000000', 'Alabama', '0', 'T', '1']; newer_row_length = 6
newer_row = ['A', 'ST0200000000000', 'Alaska', '0', 'T', '149']; newer_row_length = 6
newer_row = ['A', 'ST0400000000000', 'Arizona', '0', 'T', '193']; newer_row_length = 6
newer_row = ['A', 'ST0500000000000', 'Arkansas', '0', 'T', '257']; newer_row_length = 6
newer_row = ['A', 'ST0600000000000', 'California', '0', 'T', '383']; newer_row_length = 6
newer_row = ['A', 'ST0800000000000', 'Colorado', '0', 'T', '772']; newer_row_length = 6
newer_row = ['A', 'ST0900000000000', 'Connecticut', '0', 'T', '908']; newer_row_length = 6
newer_row = ['A', 'ST1000000000000', 'Delaware', '0', 'T', '1100']; newer_row_length = 6
newer_row = ['A', 'ST1100000000000', 'District', 'of', 'Columbia', '0', 'T', '1112']; newer_row_length = 8
newer_row = ['A', 'ST1200000000000', 'Florida', '0', 'T'

,Raw Data,Area Type Code,City Area Code,City
0,﻿area_type_code area_code area_text ...,﻿area_type_code,area_code,
1,A ST0100000000000 Alabama 0 T 1,A,ST0100000000000,
2,A ST0200000000000 Alaska 0 T ...,A,ST0200000000000,
3,A ST0400000000000 Arizona 0 T ...,A,ST0400000000000,
4,A ST0500000000000 Arkansas 0 ...,A,ST0500000000000,
5,A ST0600000000000 California 0 ...,A,ST0600000000000,
6,A ST0800000000000 Colorado 0 ...,A,ST0800000000000,
7,A ST0900000000000 Connecticut 0 ...,A,ST0900000000000,
8,A ST1000000000000 Delaware 0 ...,A,ST1000000000000,
9,A ST1100000000000 District of Columbia ...,A,ST1100000000000,District


In [9]:
top200_loc_pop_df = city_loc_pop_df.head(200)

top200cities_df = pd.DataFrame(columns=['City','Latitude','Longitude','Population', 'Unemployment Rate Series ID', 'Unemployment Series ID', 'Labor Force Participation Rate Series ID'])

top200cities_df['City'] = city_loc_pop_df['City']
top200cities_df['Latitude'] = city_loc_pop_df['Latitude']
top200cities_df['Longitude'] = city_loc_pop_df['Longitude']
top200cities_df['Population'] = city_loc_pop_df['Population']

top200cities_df.head(50)


,City,Latitude,Longitude,Population,Unemployment Rate Series ID,Unemployment Series ID,Labor Force Participation Rate Series ID
0,New York,40.6943,-73.9249,18972871,NaN,NaN,NaN
1,Los Angeles,34.1141,-118.4068,12121244,NaN,NaN,NaN
2,Chicago,41.8375,-87.6866,8595181,NaN,NaN,NaN
3,Miami,25.7840,-80.2101,5711945,NaN,NaN,NaN
4,Dallas,32.7935,-96.7667,5668165,NaN,NaN,NaN
5,Houston,29.7860,-95.3885,5650910,NaN,NaN,NaN
6,Philadelphia,40.0077,-75.1339,5512873,NaN,NaN,NaN
7,Atlanta,33.7628,-84.4220,5046555,NaN,NaN,NaN
8,Washington,38.9047,-77.0163,4810669,NaN,NaN,NaN
9,Boston,42.3188,-71.0852,4208580,NaN,NaN,NaN


In [10]:

top_cities_df = top200cities_df.merge(city_area_codes_df, how='left', on='City')

top_cities_df.head(10)


,City,Latitude,Longitude,Population,Unemployment Rate Series ID,Unemployment Series ID,Labor Force Participation Rate Series ID,City Area Code
0,New York,40.6943,-73.9249,18972871,NaN,NaN,NaN,CT3651000000000
1,Los Angeles,34.1141,-118.4068,12121244,NaN,NaN,NaN,CT0644000000000
2,Chicago,41.8375,-87.6866,8595181,NaN,NaN,NaN,CT1714000000000
3,Miami,25.7840,-80.2101,5711945,NaN,NaN,NaN,CT1245000000000
4,Dallas,32.7935,-96.7667,5668165,NaN,NaN,NaN,CT4819000000000
5,Houston,29.7860,-95.3885,5650910,NaN,NaN,NaN,CT4835000000000
6,Philadelphia,40.0077,-75.1339,5512873,NaN,NaN,NaN,CT4260000000000
7,Atlanta,33.7628,-84.4220,5046555,NaN,NaN,NaN,CT1304000000000
8,Washington,38.9047,-77.0163,4810669,NaN,NaN,NaN,CT1150000000000
9,Washington,38.9047,-77.0163,4810669,NaN,NaN,NaN,CT4981960000000


In [11]:

top_cities_df.drop_duplicates(subset=['City'], inplace=True)

top_cities_df.dropna(subset=['City Area Code'], inplace=True)

top_cities_df.reset_index(inplace=True)
top_cities_df.drop(columns='index', inplace=True)

top_cities_df.head(10)


,City,Latitude,Longitude,Population,Unemployment Rate Series ID,Unemployment Series ID,Labor Force Participation Rate Series ID,City Area Code
0,New York,40.6943,-73.9249,18972871,NaN,NaN,NaN,CT3651000000000
1,Los Angeles,34.1141,-118.4068,12121244,NaN,NaN,NaN,CT0644000000000
2,Chicago,41.8375,-87.6866,8595181,NaN,NaN,NaN,CT1714000000000
3,Miami,25.7840,-80.2101,5711945,NaN,NaN,NaN,CT1245000000000
4,Dallas,32.7935,-96.7667,5668165,NaN,NaN,NaN,CT4819000000000
5,Houston,29.7860,-95.3885,5650910,NaN,NaN,NaN,CT4835000000000
6,Philadelphia,40.0077,-75.1339,5512873,NaN,NaN,NaN,CT4260000000000
7,Atlanta,33.7628,-84.4220,5046555,NaN,NaN,NaN,CT1304000000000
8,Washington,38.9047,-77.0163,4810669,NaN,NaN,NaN,CT1150000000000
9,Boston,42.3188,-71.0852,4208580,NaN,NaN,NaN,CT2507000000000


In [12]:

top_cities_df['Unemployment Rate Series ID'] = 'LAU' + top_cities_df['City Area Code'] + '03'
top_cities_df['Unemployment Series ID'] = 'LAU' + top_cities_df['City Area Code'] + '04'
top_cities_df['Labor Force Participation Rate Series ID'] = 'LAU' + top_cities_df['City Area Code'] + '08'

top_cities_df.head(10)


,City,Latitude,Longitude,Population,Unemployment Rate Series ID,Unemployment Series ID,Labor Force Participation Rate Series ID,City Area Code
0,New York,40.6943,-73.9249,18972871,LAUCT365100000000003,LAUCT365100000000004,LAUCT365100000000008,CT3651000000000
1,Los Angeles,34.1141,-118.4068,12121244,LAUCT064400000000003,LAUCT064400000000004,LAUCT064400000000008,CT0644000000000
2,Chicago,41.8375,-87.6866,8595181,LAUCT171400000000003,LAUCT171400000000004,LAUCT171400000000008,CT1714000000000
3,Miami,25.7840,-80.2101,5711945,LAUCT124500000000003,LAUCT124500000000004,LAUCT124500000000008,CT1245000000000
4,Dallas,32.7935,-96.7667,5668165,LAUCT481900000000003,LAUCT481900000000004,LAUCT481900000000008,CT4819000000000
5,Houston,29.7860,-95.3885,5650910,LAUCT483500000000003,LAUCT483500000000004,LAUCT483500000000008,CT4835000000000
6,Philadelphia,40.0077,-75.1339,5512873,LAUCT426000000000003,LAUCT426000000000004,LAUCT426000000000008,CT4260000000000
7,Atlanta,33.7628,-84.4220,5046555,LAUCT130400000000003,LAUCT130400000000004,LAUCT130400000000008,CT1304000000000
8,Washington,38.9047,-77.0163,4810669,LAUCT115000000000003,LAUCT115000000000004,LAUCT115000000000008,CT1150000000000
9,Boston,42.3188,-71.0852,4208580,LAUCT250700000000003,LAUCT250700000000004,LAUCT250700000000008,CT2507000000000


In [13]:

top50df = top_cities_df.iloc[0:50]

top50df.head(10)


,City,Latitude,Longitude,Population,Unemployment Rate Series ID,Unemployment Series ID,Labor Force Participation Rate Series ID,City Area Code
0,New York,40.6943,-73.9249,18972871,LAUCT365100000000003,LAUCT365100000000004,LAUCT365100000000008,CT3651000000000
1,Los Angeles,34.1141,-118.4068,12121244,LAUCT064400000000003,LAUCT064400000000004,LAUCT064400000000008,CT0644000000000
2,Chicago,41.8375,-87.6866,8595181,LAUCT171400000000003,LAUCT171400000000004,LAUCT171400000000008,CT1714000000000
3,Miami,25.7840,-80.2101,5711945,LAUCT124500000000003,LAUCT124500000000004,LAUCT124500000000008,CT1245000000000
4,Dallas,32.7935,-96.7667,5668165,LAUCT481900000000003,LAUCT481900000000004,LAUCT481900000000008,CT4819000000000
5,Houston,29.7860,-95.3885,5650910,LAUCT483500000000003,LAUCT483500000000004,LAUCT483500000000008,CT4835000000000
6,Philadelphia,40.0077,-75.1339,5512873,LAUCT426000000000003,LAUCT426000000000004,LAUCT426000000000008,CT4260000000000
7,Atlanta,33.7628,-84.4220,5046555,LAUCT130400000000003,LAUCT130400000000004,LAUCT130400000000008,CT1304000000000
8,Washington,38.9047,-77.0163,4810669,LAUCT115000000000003,LAUCT115000000000004,LAUCT115000000000008,CT1150000000000
9,Boston,42.3188,-71.0852,4208580,LAUCT250700000000003,LAUCT250700000000004,LAUCT250700000000008,CT2507000000000


In [14]:
next50df = top_cities_df.iloc[50:100]

next50df.head(10)

,City,Latitude,Longitude,Population,Unemployment Rate Series ID,Unemployment Series ID,Labor Force Participation Rate Series ID,City Area Code
50,Tucson,32.1541,-110.8787,875284,LAUCT047700000000003,LAUCT047700000000004,LAUCT047700000000008,CT0477000000000
51,Honolulu,21.3294,-157.8460,835291,LAUCT151699900000003,LAUCT151699900000004,LAUCT151699900000008,CT1516999000000
52,McAllen,26.2252,-98.2467,809002,LAUCT484538400000003,LAUCT484538400000004,LAUCT484538400000008,CT4845384000000
53,Omaha,41.2627,-96.0529,806485,LAUCT313700000000003,LAUCT313700000000004,LAUCT313700000000008,CT3137000000000
54,El Paso,31.8476,-106.4300,794344,LAUCT482400000000003,LAUCT482400000000004,LAUCT482400000000008,CT4824000000000
55,Albuquerque,35.1054,-106.6465,765693,LAUCT350200000000003,LAUCT350200000000004,LAUCT350200000000008,CT3502000000000
56,Rochester,43.1680,-77.6162,737309,LAUCT275488000000003,LAUCT275488000000004,LAUCT275488000000008,CT2754880000000
57,Sarasota,27.3387,-82.5432,727388,LAUCT126417500000003,LAUCT126417500000004,LAUCT126417500000008,CT1264175000000
58,Fresno,36.7830,-119.7939,719558,LAUCT062700000000003,LAUCT062700000000004,LAUCT062700000000008,CT0627000000000
59,Tulsa,36.1283,-95.9042,715983,LAUCT407500000000003,LAUCT407500000000004,LAUCT407500000000008,CT4075000000000


In [15]:

# Unemployment rate top 50 data pull

top50_unemployment_rate_json_params = {"seriesid":top50df['Unemployment Rate Series ID'].tolist(), "catalog":"false", "startyear":"2010", "endyear":"2023", "registrationkey":bls_api_key}
top50_unemployment_rate_df = pd.DataFrame(columns=['Unemployment Rate Series ID','August 2023 Unemployment Rate'])


headers = {'Content-type': 'application/json'}
data = json.dumps(top50_unemployment_rate_json_params)
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)

### print(json_data)


seriesIDs_list = []
latest_values_list = []


for series in json_data['Results']['series']:
    
    seriesID = series['seriesID']
    seriesIDs_list.append(seriesID)
    
    latest_value = series['data'][0]['value']
    latest_values_list.append(latest_value)

    ### print(f"series ID: {seriesID},latest value:{latest_value}")


top50_unemployment_rate_df['Unemployment Rate Series ID'] = seriesIDs_list
top50_unemployment_rate_df['August 2023 Unemployment Rate'] = latest_values_list


top50_unemployment_rate_df.head()


### for looping through more data points from each series for timeseries data for individual cities (maybe/later)    
#    for item in series['data']:
#        year = item['year']
#        period = item['period']
#        value = item['value']


,Unemployment Rate Series ID,August 2023 Unemployment Rate
0,LAUCT365100000000003,5.6
1,LAUCT064400000000003,5.7
2,LAUCT171400000000003,4.4
3,LAUCT124500000000003,1.8
4,LAUCT481900000000003,4.3


In [16]:

# Unemployment rate next 50 data pull

next50_unemployment_rate_json_params = {"seriesid":next50df['Unemployment Rate Series ID'].tolist(), "catalog":"false", "startyear":"2010", "endyear":"2023", "registrationkey":bls_api_key}
next50_unemployment_rate_df = pd.DataFrame(columns=['Unemployment Rate Series ID','August 2023 Unemployment Rate'])


headers = {'Content-type': 'application/json'}
data = json.dumps(next50_unemployment_rate_json_params)
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)

### print(json_data)


seriesIDs_list = []
latest_values_list = []

for series in json_data['Results']['series']:
    
    seriesID = series['seriesID']
    seriesIDs_list.append(seriesID)
    
    latest_value = series['data'][0]['value']
    latest_values_list.append(latest_value)

    ### print(f"series ID: {seriesID},latest value:{latest_value}")


next50_unemployment_rate_df['Unemployment Rate Series ID'] = seriesIDs_list
next50_unemployment_rate_df['August 2023 Unemployment Rate'] = latest_values_list


next50_unemployment_rate_df.head()


### for looping through more data points from each series for timeseries data for individual cities (maybe/later)    
#    for item in series['data']:
#        year = item['year']
#        period = item['period']
#        value = item['value']


,Unemployment Rate Series ID,August 2023 Unemployment Rate
0,LAUCT047700000000003,4.9
1,LAUCT151699900000003,2.6
2,LAUCT484538400000003,5.1
3,LAUCT313700000000003,2.6
4,LAUCT482400000000003,4.9


In [17]:
# combined unemployment rate data for top 100 US cities

unemployment_rate_df = pd.concat([top50_unemployment_rate_df,next50_unemployment_rate_df], ignore_index=True)

unemployment_rate_df.head()


,Unemployment Rate Series ID,August 2023 Unemployment Rate
0,LAUCT365100000000003,5.6
1,LAUCT064400000000003,5.7
2,LAUCT171400000000003,4.4
3,LAUCT124500000000003,1.8
4,LAUCT481900000000003,4.3


In [18]:

# Unemployment (abs) top 50 data pull

top50_unemployment_json_params = {"seriesid":top50df['Unemployment Series ID'].tolist(), "catalog":"false", "startyear":"2010", "endyear":"2023", "registrationkey":bls_api_key}
top50_unemployment_df = pd.DataFrame(columns=['Unemployment Series ID','August 2023 Unemployment'])


headers = {'Content-type': 'application/json'}
data = json.dumps(top50_unemployment_json_params)
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)

### print(json_data)


seriesIDs_list = []
latest_values_list = []


for series in json_data['Results']['series']:
    
    seriesID = series['seriesID']
    seriesIDs_list.append(seriesID)
    
    latest_value = series['data'][0]['value']
    latest_values_list.append(latest_value)

    ### print(f"series ID: {seriesID},latest value:{latest_value}")


top50_unemployment_df['Unemployment Series ID'] = seriesIDs_list
top50_unemployment_df['August 2023 Unemployment'] = latest_values_list


top50_unemployment_df.head()


### for looping through more data points from each series for timeseries data for individual cities (maybe/later)    
#    for item in series['data']:
#        year = item['year']
#        period = item['period']
#        value = item['value']


,Unemployment Series ID,August 2023 Unemployment
0,LAUCT365100000000004,234715
1,LAUCT064400000000004,117804
2,LAUCT171400000000004,61024
3,LAUCT124500000000004,4350
4,LAUCT481900000000004,32665


In [19]:

# Unemployment (abs) next 50 data pull

next50_unemployment_json_params = {"seriesid":next50df['Unemployment Series ID'].tolist(), "catalog":"false", "startyear":"2010", "endyear":"2023", "registrationkey":bls_api_key}
next50_unemployment_df = pd.DataFrame(columns=['Unemployment Series ID','August 2023 Unemployment'])


headers = {'Content-type': 'application/json'}
data = json.dumps(next50_unemployment_json_params)
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)

### print(json_data)


seriesIDs_list = []
latest_values_list = []


for series in json_data['Results']['series']:
    
    seriesID = series['seriesID']
    seriesIDs_list.append(seriesID)
    
    latest_value = series['data'][0]['value']
    latest_values_list.append(latest_value)

    ### print(f"series ID: {seriesID},latest value:{latest_value}")


next50_unemployment_df['Unemployment Series ID'] = seriesIDs_list
next50_unemployment_df['August 2023 Unemployment'] = latest_values_list


next50_unemployment_df.head()


### for looping through more data points from each series for timeseries data for individual cities (maybe/later)    
#    for item in series['data']:
#        year = item['year']
#        period = item['period']
#        value = item['value']


,Unemployment Series ID,August 2023 Unemployment
0,LAUCT047700000000004,13077
1,LAUCT151699900000004,12026
2,LAUCT484538400000004,3635
3,LAUCT313700000000004,6780
4,LAUCT482400000000004,15561


In [20]:
# combined unemployment data for top 100 US cities

unemployment_df = pd.concat([top50_unemployment_df,next50_unemployment_df], ignore_index=True)

unemployment_df.head()


,Unemployment Series ID,August 2023 Unemployment
0,LAUCT365100000000004,234715
1,LAUCT064400000000004,117804
2,LAUCT171400000000004,61024
3,LAUCT124500000000004,4350
4,LAUCT481900000000004,32665


In [21]:

top_cities_combined_df = top_cities_df.merge(unemployment_rate_df, how='inner', on='Unemployment Rate Series ID').merge(unemployment_df, how='inner', on='Unemployment Series ID')

print(f"# of rows/cities: {len(top_cities_combined_df.index)}")

top_cities_combined_df.head()


# of rows/cities: 100


,City,Latitude,Longitude,Population,Unemployment Rate Series ID,Unemployment Series ID,Labor Force Participation Rate Series ID,City Area Code,August 2023 Unemployment Rate,August 2023 Unemployment
0,New York,40.6943,-73.9249,18972871,LAUCT365100000000003,LAUCT365100000000004,LAUCT365100000000008,CT3651000000000,5.6,234715
1,Los Angeles,34.1141,-118.4068,12121244,LAUCT064400000000003,LAUCT064400000000004,LAUCT064400000000008,CT0644000000000,5.7,117804
2,Chicago,41.8375,-87.6866,8595181,LAUCT171400000000003,LAUCT171400000000004,LAUCT171400000000008,CT1714000000000,4.4,61024
3,Miami,25.7840,-80.2101,5711945,LAUCT124500000000003,LAUCT124500000000004,LAUCT124500000000008,CT1245000000000,1.8,4350
4,Dallas,32.7935,-96.7667,5668165,LAUCT481900000000003,LAUCT481900000000004,LAUCT481900000000008,CT4819000000000,4.3,32665


In [22]:
city_data_df = top_cities_combined_df.loc[:, ['City','Latitude','Longitude','Population','August 2023 Unemployment Rate','August 2023 Unemployment']]

city_data_df.rename(columns={'City':'city','Latitude':'latitude','Longitude':'longitude','Population':'population','August 2023 Unemployment Rate':'unemploymentRate','August 2023 Unemployment':'unemploymentCount'},inplace=True)

city_data_df.head()

,city,latitude,longitude,population,unemploymentRate,unemploymentCount
0,New York,40.6943,-73.9249,18972871,5.6,234715
1,Los Angeles,34.1141,-118.4068,12121244,5.7,117804
2,Chicago,41.8375,-87.6866,8595181,4.4,61024
3,Miami,25.7840,-80.2101,5711945,1.8,4350
4,Dallas,32.7935,-96.7667,5668165,4.3,32665


In [23]:

engine = create_engine('sqlite:///city_db.sqlite', echo=True)
sqlite_connection = engine.connect()
sqlite_table = "CityUnemployment"


In [25]:

city_data_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace')


2023-09-25 13:20:25,490 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-25 13:20:25,493 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CityUnemployment")
2023-09-25 13:20:25,493 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-25 13:20:25,494 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("CityUnemployment")
2023-09-25 13:20:25,495 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-25 13:20:25,495 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2023-09-25 13:20:25,496 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-25 13:20:25,496 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("CityUnemployment")
2023-09-25 13:20:25,497 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-25 13:20:25,498 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-09-25 13:20:25,499 INFO sqlalchemy.engine.Engine [raw sql] ('CityU

100

In [26]:

sqlite_connection.close()
